In [23]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import NNConv

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [25]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [26]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [27]:
file = dataDir + dataFile
dataRaw = loadData(file)

Raw shape:  (7374816, 5)
Days:  638


In [28]:
# dataIn =  pd.pivot_table(dataRaw, values='amount', index=['date'],
#                     columns=['end_id'], aggfunc=np.sum, fill_value=0)
# dataIn.head(1)

In [29]:
dataRaw.head(2)

,Unnamed: 0,start_id,end_id,date,amount
0,0,BL01,BL01,2017-01-01,70.0
1,1,BL01,BL01,2017-01-02,40.0


In [30]:
# dataOut =  pd.pivot_table(dataRaw, values='amount', index=['date'],
#                     columns=['start_id'], aggfunc=np.sum, fill_value=0)
# dataOut.head(1)

In [31]:
# matrix = np.stack((dataOut.values, dataIn.values),-1)
# matrix.shape

In [32]:
dataEdge = getTimeSeries(dataRaw)
#matrix = dataEdge.values
matrix = np.load('communityAggregatedMatrix.npy')
matrix = matrix.astype(float)
matrix.shape

(638, 100)

In [33]:
# for i in range(matrix.shape[1]):
#         matrix[:, i] = (matrix[:, i] - np.mean(matrix[:, i])) / (np.std(matrix[:, i]))

In [34]:
for i in range(matrix.shape[0]):
        matrix[i,:] = (matrix[i,:] - np.mean(matrix[i,:])) / (np.std(matrix[i,:]))

In [35]:
stations = list(set(dataRaw.start_id))
n= 10 #len(stations)
edge_index = [[a//n,a%n] for a in range(n*n)]
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [36]:
dates = list(dataEdge.index)
DOW = list(pd.to_datetime(dataEdge.index.values).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [37]:
dataList = []

for i in range(len(DOW)):
    x = torch.tensor(np.ones((n,1)), dtype=torch.float)
    y = torch.tensor(np.array([DOW[i]]), dtype=torch.long)
    e = torch.tensor(matrix[i].reshape(matrix.shape[1],1), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y,edge_attr=e)
    dataList.append(data)

dataList[0]

Data(edge_attr=[100, 1], edge_index=[2, 100], x=[10, 1], y=[1])

In [38]:
n_classes = 2
featureDim = 1
hiddenDim = [8,16,20]
num_edge_features = 1

In [39]:
# NNConv?

In [40]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = GCNConv(featureDim, hiddenDim[0])
        self.conv1Lin = nn.Linear(1, featureDim * hiddenDim[0])
        self.conv1 = NNConv(featureDim, hiddenDim[0],self.conv1Lin)
        #self.conv1 = NNConv(featureDim, hiddenDim[0],nn.Linear(1, 1000))
        
        #self.conv2 = GCNConv(hiddenDim[0], hiddenDim[1])
        self.conv2Lin = nn.Linear(1, hiddenDim[0]*hiddenDim[1])
        self.conv2 = NNConv(hiddenDim[0], hiddenDim[1],self.conv2Lin)
        
        self.linear1 = nn.Linear(n*hiddenDim[1], hiddenDim[2])
        self.linear2 = nn.Linear(hiddenDim[2], n_classes)

        self.init_weights(self.linear1)
        self.init_weights(self.linear2)
        self.init_weights(self.conv1Lin)
        self.init_weights(self.conv2Lin)
        
    def forward(self, data):
        x, edge_index,edge_attr = data.x, data.edge_index, data.edge_attr
        #print(x.shape)
        #print(x[0])
        x = self.conv1(x, edge_index,edge_attr)
        #print(x.shape)
        #print(x[0])
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index,edge_attr)
        #print(x.shape)
        x = F.relu(x)
        x = x.view(-1,n*hiddenDim[1])
        #print(x.shape)
        x = self.linear1(x)
        #print(x.shape)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.linear2(x)
        #print(x.shape)
        return F.log_softmax(x, dim=1)
        
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x, edge_index,edge_attr = data.x, data.edge_index,data.edge_attr
        x = self.conv1(x, edge_index,edge_attr)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index,edge_attr)
        x = F.relu(x)
        x = x.view(-1,n*hiddenDim[1])
        x = self.linear1(x)
        return x
            

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = Net().to(device)
#data = dataset[0].to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [42]:
device

device(type='cuda')

In [43]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
num_epochs = 200

In [44]:
model.train()

for epoch in range(num_epochs):
    epochLoss = 0
    for data in dataList:
        #data.view(1,-1,-1)
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
#         print(out.shape)
#         print(data.y.shape)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        epochLoss += loss.item()
    
#     if epoch == 500:
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=5e-4)
        
    print('epoch [{}/{}], loss:{:.4f}'
    .format(epoch + 1, num_epochs, np.mean(epochLoss)))

epoch [1/200], loss:388.4520
epoch [2/200], loss:184.7511
epoch [3/200], loss:147.3527
epoch [4/200], loss:130.9510
epoch [5/200], loss:119.4655
epoch [6/200], loss:112.8412
epoch [7/200], loss:108.5162
epoch [8/200], loss:104.9295
epoch [9/200], loss:101.8394
epoch [10/200], loss:99.2785
epoch [11/200], loss:96.6827
epoch [12/200], loss:95.3531
epoch [13/200], loss:93.4233
epoch [14/200], loss:91.8623
epoch [15/200], loss:90.3995
epoch [16/200], loss:88.7728
epoch [17/200], loss:88.7954
epoch [18/200], loss:87.2309
epoch [19/200], loss:86.1458
epoch [20/200], loss:85.2580
epoch [21/200], loss:84.6035
epoch [22/200], loss:83.4948
epoch [23/200], loss:83.6350
epoch [24/200], loss:82.7301
epoch [25/200], loss:82.2469
epoch [26/200], loss:81.7070
epoch [27/200], loss:81.3029
epoch [28/200], loss:81.0326
epoch [29/200], loss:80.4590
epoch [30/200], loss:80.2800
epoch [31/200], loss:79.6748
epoch [32/200], loss:79.3099
epoch [33/200], loss:79.2315
epoch [34/200], loss:78.5355
epoch [35/200]

In [45]:
model.eval()
with torch.no_grad():
    y = DOW
    predList = []
    for data in dataList:
        data = data.to(device)
        pred = model(data).cpu().numpy()
        predList.append(np.argmax(pred))
    
    print('Accuracy: {:.4f}'.format( np.sum(np.array(predList) == np.array(y)) / len(predList) ))

Accuracy: 0.9734


In [46]:
representation = np.empty((matrix.shape[0],hiddenDim[2]))
#representationR = np.empty((matrix.shape[0],hiddenDim[2]))
prediction = np.empty((matrix.shape[0],n_classes))
model.eval()
with torch.no_grad():
    for i,data in enumerate(dataList):
        data = data.to(device)
        representation[i] = model.representation(data).cpu().numpy()
        #representationR[i] = model.representationR(data).cpu().numpy()
        prediction[i] = model(data).cpu().numpy()
        predList.append(np.argmax(pred))

In [47]:
representation.shape

(638, 20)

In [48]:
prediction.shape

(638, 2)

In [49]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [50]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [51]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [52]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = [str(d.date()) for d in dates]

In [53]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [54]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [55]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [56]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [57]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
T

In [58]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.228571,0.003284,0.666667,0.137931,0.01
1,Global,0.500000,0.011494,0.631579,0.413793,0.03
2,Global,0.444444,0.021346,0.480000,0.413793,0.04
3,Global,0.500000,0.026273,0.483871,0.517241,0.05
4,Global,0.567164,0.031199,0.500000,0.655172,0.06
5,Global,0.547945,0.039409,0.454545,0.689655,0.07
6,Global,0.500000,0.050903,0.392157,0.689655,0.08
7,Global,0.500000,0.065681,0.365079,0.793103,0.10
8,Global,0.346154,0.164204,0.212598,0.931034,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30
